In [1]:
#!pip install celery

In [2]:
from test import echo

In [3]:
res1 = echo.delay('Python rocks 1 :) !'); print(type(res1)); print(res1)

<class 'celery.result.AsyncResult'>
5df4da2c-4b9a-4a61-a6ad-51918a9d9368


In [4]:
res1.ready()

True

In [5]:
res1.result

'Python rocks 1 :) !'

In [6]:
res2 = echo.delay('Python rocks 2 :) !'); print(type(res2)); print(res2)
res2.ready()
res2.result

<class 'celery.result.AsyncResult'>
8b0579f4-86b9-42bf-a663-1b275ad74e0e
